ETL

In [28]:
# Se importan las librerias usadas en el código
import pandas as pd
import numpy as np

In [29]:
data=pd.read_csv(r"D:\Programacion\DataScience_Henry\PI1\Dataset\movies_dataset.csv")

C:\Users\lasso\AppData\Local\Temp\ipykernel_21440\2189959564.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv(r"D:\Programacion\DataScience_Henry\PI1\Dataset\movies_dataset.csv")


In [30]:
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [31]:
data.drop(columns=["video","imdb_id","adult","original_title","poster_path" , "homepage"],inplace=True)

In [32]:
data.loc[data["revenue"].isna(),"revenue"]=0

In [33]:
data.loc[data["budget"].isna(),"budget"]=0

In [34]:
data.dropna(subset=["release_date"],inplace=True)

In [35]:
# Quedan bastantes datos nulos, pero como en las transformaciones nos dicen de hacer nada más que esos puntos señalados
# No se tocan.
data.isna().sum()

belongs_to_collection    40888
budget                       0
genres                       0
id                           0
original_language           11
overview                   941
popularity                   2
production_companies         0
production_countries         0
release_date                 0
revenue                      0
runtime                    249
spoken_languages             3
status                      83
tagline                  24981
title                        3
vote_average                 3
vote_count                   3
dtype: int64

In [36]:
data["belongs_to_collection"][0]

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

Creo la columna "return"

In [37]:
# Verifico que sean de un tipo de dato divisible

print(type(data["revenue"][0]))
print(type(data["budget"][0]))
# La columna budget no puede simplemente pasarse a float ya que contiene valores que son string

<class 'numpy.float64'>
<class 'str'>


In [38]:
data["budget"]=pd.to_numeric(data["budget"],errors="coerce")
print(data["budget"].values)
print(data["budget"].unique())

[30000000. 65000000.        0. ...        0.        0.        0.]
[30000000. 65000000.        0. ...  3417000. 25868826.  1254040.]


In [40]:
# Ahora la idea es hacer la división pero con un try and except
# donde si la división va a tirar un error, sea porque budget es cero(Estará correcto este caso?) o string,
# se le deja el valor cero a la nueva columna.
columna=[]
for i, j in zip(data["revenue"], data["budget"]):
    if j!=0:
        try:
            columna.append(i/j)
        except:
            columna.append(0)
    else:
        columna.append(0)

# Agrego la columna "revenue"
data["return"]=columna

In [45]:
data["release_date"].unique()

array(['1995-10-30', '1995-12-15', '1995-12-22', ..., '1904-03-05',
       '1991-05-13', '1917-10-21'], dtype=object)

In [46]:
print(data["release_date"].dtype)

object


In [50]:
print(data["release_date"].unique())

[Timestamp('1995-10-30 00:00:00') Timestamp('1995-12-15 00:00:00')
 Timestamp('1995-12-22 00:00:00') ... '2003-08-01' '1917-10-21'
 '2017-06-09']


In [50]:
print(data["release_date"].unique())

[Timestamp('1995-10-30 00:00:00') Timestamp('1995-12-15 00:00:00')
 Timestamp('1995-12-22 00:00:00') ... '2003-08-01' '1917-10-21'
 '2017-06-09']


In [52]:
data["release_date"][10394]

Timestamp('2005-07-14 00:00:00')

In [49]:
data["release_date"].shape

(45379,)